In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

glueContext = GlueContext(SparkContext.getOrCreate())

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
12,application_1632897281066_0013,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
salesDF = glueContext.create_dynamic_frame.from_catalog(
             database="johndb",
             table_name="sales")
salesDF = salesDF.toDF();
salesDF.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------+----------+
|ordernumber|quantityordered|priceeach|orderlinenumber|  sales|      orderdate| status|qtr_id|month_id|year_id|productline|msrp|productcode|dealsize|customerid|
+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------+----------+
|      10107|             30|     95.7|              2| 2871.0| 2/24/2003 0:00|Shipped|     1|       2|   2003|Motorcycles|  95|   S10_1678|   Small|         1|
|      10121|             34|    81.35|              5| 2765.9|  5/7/2003 0:00|Shipped|     2|       5|   2003|Motorcycles|  95|   S10_1678|   Small|         2|
|      10134|             41|    94.74|              2|3884.34|  7/1/2003 0:00|Shipped|     3|       7|   2003|Motorcycles|  95|   S10_1678|  Medium|         3|
|      10145|             45|    8

In [4]:
salesDF.crosstab('productline', 'dealsize').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+------+-----+
|productline_dealsize|Large|Medium|Small|
+--------------------+-----+------+-----+
|         Motorcycles|   19|   156|  156|
|              Trains|    1|    26|   50|
|        Classic Cars|   97|   530|  340|
|              Planes|    9|   129|  168|
|    Trucks and Buses|    7|   175|  119|
|               Ships|    0|   107|  127|
|        Vintage Cars|   24|   261|  322|
+--------------------+-----+------+-----+

In [3]:
salesDF.describe('sales').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|summary|             sales|
+-------+------------------+
|  count|              2823|
|   mean|  3553.88907190932|
| stddev|1841.8651057401842|
|    min|            482.13|
|    max|           14082.8|
+-------+------------------+

In [4]:
salesDF.groupby('productline').count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----+
|     productline|count|
+----------------+-----+
|     Motorcycles|  331|
|    Vintage Cars|  607|
|           Ships|  234|
|Trucks and Buses|  301|
|    Classic Cars|  967|
|          Trains|   77|
|          Planes|  306|
+----------------+-----+

In [7]:
salesDF.select("productline","quantityordered").groupby('productline').min().show()
salesDF.select("productline","quantityordered").groupby('productline').max().show()
salesDF.select("productline","quantityordered").groupby('productline').mean().show()
salesDF.select("productline","quantityordered").groupby('productline').sum().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+--------------------+
|     productline|min(quantityordered)|
+----------------+--------------------+
|     Motorcycles|                  20|
|    Vintage Cars|                   6|
|           Ships|                  20|
|Trucks and Buses|                  11|
|    Classic Cars|                  10|
|          Trains|                  20|
|          Planes|                  11|
+----------------+--------------------+

+----------------+--------------------+
|     productline|max(quantityordered)|
+----------------+--------------------+
|     Motorcycles|                  66|
|    Vintage Cars|                  76|
|           Ships|                  55|
|Trucks and Buses|                  70|
|    Classic Cars|                  97|
|          Trains|                  51|
|          Planes|                  85|
+----------------+--------------------+

+----------------+--------------------+
|     productline|avg(quantityordered)|
+----------------+--------------------

In [8]:
salesDF.groupby('productline').agg({'quantityordered': 'mean','sales': 'sum','priceeach' : 'max'}).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+--------------+--------------------+------------------+
|     productline|max(priceeach)|avg(quantityordered)|        sum(sales)|
+----------------+--------------+--------------------+------------------+
|     Motorcycles|         100.0|  35.235649546827794|1166388.3400000003|
|    Vintage Cars|         100.0|   34.71004942339374|1903150.8399999992|
|           Ships|         100.0|   34.73076923076923|         714437.13|
|Trucks and Buses|         100.0|   35.80398671096346|1127789.8399999996|
|    Classic Cars|         100.0|  35.152016546018615| 3919615.659999997|
|          Trains|         100.0|   35.22077922077922|226243.46999999997|
|          Planes|         100.0|   35.05555555555556| 975003.5700000001|
+----------------+--------------+--------------------+------------------+

In [9]:
salesDF.orderBy(salesDF.sales).show()
salesDF.orderBy(salesDF.sales.desc()).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------------+---------+---------------+------+---------------+----------+------+--------+-------+----------------+----+-----------+--------+----------+
|ordernumber|quantityordered|priceeach|orderlinenumber| sales|      orderdate|    status|qtr_id|month_id|year_id|     productline|msrp|productcode|dealsize|customerid|
+-----------+---------------+---------+---------------+------+---------------+----------+------+--------+-------+----------------+----+-----------+--------+----------+
|      10425|             11|    43.83|              6|482.13| 5/31/2005 0:00|In Process|     2|       5|   2005|Trucks and Buses|  54|   S32_2509|   Small|      2250|
|      10407|              6|    90.19|              3|541.14| 4/22/2005 0:00|   On Hold|     2|       4|   2005|    Vintage Cars|  92|   S18_4409|   Small|      1288|
|      10408|             15|    36.93|              1|553.95| 4/22/2005 0:00|   Shipped|     2|       4|   2005|    Vintage Cars|  41|   S24_3969|   Small|    